In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.functions import softmax
from models.rnnlm import Rnnlm
from models.better_rnnlm import BetterRnnlm

In [2]:
# Implementing sentence generation with RNN
class RNNLMGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

In [6]:
# Using Rnnlm.pkl pretrained model from previous chapter
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RNNLMGen()
model.load_params('./models/Rnnlm.pkl')

# Setting 'Start' and 'Skip' word
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# Generating sentence
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you need a pressure to top supply and between government capital markets.
 mr. stone 's store 's case the cost for criminal election south gardens all mr. honecker out of the complaint designed to make the communist party of merge into third world.
 mr. honecker this morning at one lower was named a former bond told it at the clients said mr. roman.
 it 's understood to find soviet scowcroft that she was young the most successful claims the coalition of abortion or an expectation here gray.
 some observers delay perlman hoped says mr. honecker.



In [10]:
# Add two numbers with seq2seq model
from dataset import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)


In [ ]:
# Implementing seq2seq model
# p.308